# Template Notebook for testing Genetic Algorithms exploring the prompt embedding latent space

In [ ]:
from evolutionary.plotting import plot_fitness_statistics
from model_helpers.auto_pipeline import auto_diffusion_pipeline
from diffusers.utils import logging
from evolutionary.image_processing import create_animation_from_generations, create_generation_image_grid, save_images_from_generation

In [ ]:
# These are diverse prompts that can be used to generate the initial population
prompts = [
    "Futuristic cityscape under a dual sunset",
    "Surreal garden with glow-in-the-dark plants",
    "Underwater scene featuring a coral-built city",
    "Robot and medieval knight engaged in a chess match",
    "Portrait composed entirely of musical instruments",
    "Bustling Martian market scene",
    "Giant tree hosting houses and shops on its branches",
    "Abstract representation embodying the concept of time",
    "Village with all glass-made houses",
    "Victorian city populated by steampunk animals",
    "Floating books replacing shelves in a library",
    "Desert landscape with blue sand and purple cacti",
    "Modern city blending ancient Roman architectural styles",
    "Fantasy world map with inverted land and sky",
    "Post-apocalyptic nature reclaiming an urban landscape",
    "Moon's surface featuring an ice cream truck",
    "Sky parade with floating ships",
    "Regal portrait of a cat in 18th-century monarch attire",
    "Dolphins swimming through cloud formations",
    "Egyptian pyramids merged with futuristic tech",
    "Mystical forest illuminated by bioluminescent flora and fauna",
    "Underwater jazz concert spectacle",
    "Storm painting crafted from various musical instruments",
    "Chessboard with miniature city pieces",
    "Surreal transition of books into birds",
    "Secret village nestled inside a volcano",
    "Computer interior reimagined as an urban landscape",
    "Amusement park themed around candy delights",
    "Space-bound art gallery",
    "Cyberpunk twist on the Renaissance era",
    "Crystal-based plant life in a garden setting",
    "Ancient civilization thriving on an alien planet",
    "Wildlife and urban landscapes in a harmonious blend",
    "Single landscape depicting all four seasons",
    "Pirate and flying shark battle scene",
    "Utopian society concept floating on a cloud",
    "Fantasy scene where shadows are colorful",
    "Water-constructed buildings in a city",
    "Futuristic tech hidden within an ancient temple",
    "Giant book with each page being a different world",
    "Ocean and sky swapping places in a landscape",
    "Renaissance noble portrait of an alien",
    "Tree growing miniature cities as fruits",
    "Mountain range resembling sleeping dragons",
    "Market specializing in bottled dreams",
    "Concert uniting classical musicians with rock stars",
    "Planetary landscape with visible ringed sky",
    "Cityscape showcasing buildings from various historical eras",
    "Watercolor-animated animals in a surreal setting",
    "Ancient ruins interspersed with holographic projections",
    'Abandoned amusement park overrun by nature, with rides encased in vines',
    'High-speed chase between hover cars in a neon-lit megastructure',
    'Interstellar zoo featuring cosmic beasts from different galaxies',
    'Subterranean city lit by bioluminescent fungi and crystals',
    'Aerial view of a city where roads and canals are intertwined',
    'Dystopian world where giant screens display the only source of sunlight',
    'Ice castle on a cloud, accessible only by flying creatures',
    'Surreal dreamscape with floating islands connected by rope bridges',
    'Wasteland with a lone, vibrant tree sheltering a myriad of species',
    'Steampunk laboratory with intricate machinery powered by steam',
    'Galactic fair with aliens trading stars and comets',
    'A library where each book opens a portal to another universe',
    'Renaissance fair in a futuristic setting, blending past and future',
    'Garden maze made of mirrors reflecting infinite skies',
    'Night market selling potions and charms under a bioluminescent moon',
    'Arctic village on the back of a giant, wandering polar bear',
    'Volcanic eruption revealing an ancient, untouched civilization',
    'Opera house where the performers are ghosts, singing untold histories',
    'Time-frozen battlefield with statuesque soldiers from different eras',
    'Oceanic abyss where shipwrecks form the architecture of an underwater city',
    'Skyline where each building is a musical instrument, playing in the wind',
    'Circus train traveling through dimensional portals, showcasing interdimensional acts',
    'Haunted mansion where each room leads to a different point in time',
    'Festival of lights where the sky is filled with glowing kites at night',
    'Underground network of caves filled with ancient, mechanical beasts',
    'A clockwork universe where planets are interconnected gears',
    'Enchanted forest with trees that whisper secrets of the universe',
    'City where gravity shifts, allowing for multidimensional architecture',
    'A dystopian society where memories are currency',
    'Rainbow waterfall cascading into a hidden elfin city',
    'Battlefield of chess with live mythical creatures as pieces',
    'A colony on a comet, farming stardust',
    'Post-apocalyptic world where technology is powered by magic',
    'Snow globe village that experiences eternal winter',
    'Grand canyon filled with skyscraper-sized totem poles',
    'A world where shadows tell different stories than their objects',
    'Museum of extinct species, with lifelike holograms',
    'A city built inside the ribcage of a giant creature',
    'A festival where balloons carry entire houses into the sky',
    'A secret society meeting in the catacombs beneath an ancient city',
    'A desert oasis that is actually a mirage hiding an advanced civilization',
    'An island where every grain of sand is a different color',
    'A palace made entirely of reflective obsidian, showing infinite reflections',
    'A space elevator connecting Earth to a space station in geostationary orbit',
    'A train that travels through the center of the Earth',
    'An ancient library buried under the desert, guarded by sphinxes',
    'A city built on the back of a giant, wandering tortoise',
    'A space colony on Jupiter’s moon, Europa, under the ice',
    'An art gallery where the paintings come to life at night',
    'A world where plants are the dominant species, and cities are alive'
]

print(len(prompts)) # Should be at least as big as the population size

In [ ]:
logging.disable_progress_bar() # Or else your output will be full of progress bars
logging.set_verbosity_error() # Enable again if you are having problems

# Factory for loading the DiffusionPipeline from the HuggingFace Hub
def setup_pipeline():
    pipe = auto_diffusion_pipeline("stabilityai/sd-turbo") # This has to be compatible with the ImageCreator
    pipe.set_progress_bar_config(disable=True)
    return pipe

def save_images_post_evaluation(g, a):
    save_images_from_generation(a.population, g)

In [ ]:
from evolutionary.prompt_encoding.image_creation import SDPromptEmbeddingImageCreator
from evolutionary.prompt_encoding.variation import \
    UniformGaussianMutatorArguments, UniformGaussianMutator, ArithmeticCrossover
from evolutionary.selectors import TournamentSelector
from evolutionary.algorithms.ga import GeneticAlgorithm
from evolutionary.image_evaluators import AestheticsImageEvaluator

population_size = 100
num_generations = 200
batch_size = 1
elitism = 1
inference_steps = 3

# Create the necessary components for the genetic algorithm
creator = SDPromptEmbeddingImageCreator(pipeline_factory=setup_pipeline, batch_size=batch_size, inference_steps=inference_steps)
evaluator = AestheticsImageEvaluator() 
crossover = ArithmeticCrossover(0.5)
# clamp_range was evaluated with pre-testing/clamp_range/sd_turbo.py
mutation_arguments = UniformGaussianMutatorArguments(mutation_rate=0.1, mutation_strength=0.5, clamp_range=(-10.3, 15.65)) 
mutator = UniformGaussianMutator(mutation_arguments)
selector = TournamentSelector(tournament_size=3)

# Prepare initial arguments
init_args = [creator.arguments_from_prompt(prompts[i]) for i in range(population_size)]

# Create and run the genetic algorithm
ga = GeneticAlgorithm(
    population_size=population_size,
    num_generations=num_generations,
    solution_creator=creator,
    evaluator=evaluator,
    mutator=mutator,
    crossover=crossover,
    selector=selector,
    initial_arguments=init_args,
    elitism_count=elitism,
    post_evaluation_callback=save_images_post_evaluation,
)

In [ ]:
best_solution = ga.run()

In [ ]:
from diffusers.utils import make_image_grid

# Show best solution
print(best_solution.fitness)
make_image_grid(best_solution.result.images, 1, batch_size)

## Visualize the evolution

In [ ]:
for gen in range(num_generations):
    create_generation_image_grid(gen, max_images=10)

In [ ]:
video_loc = create_animation_from_generations(num_generations)
print(video_loc)
from IPython.display import Video
Video(filename=video_loc) # This does not work for all browsers/notebooks, set embed to true when you want to include it in the notebook (warning large file size)

## Plot fitness statistics

In [ ]:
plot_fitness_statistics(num_generations, ga.best_fitness, ga.worst_fitness, ga.avg_fitness)

## Save notebook and components

In [ ]:
!jupyter nbconvert --to html ga_notebook.ipynb

### Save the run to disk

In [ ]:
import pickle
import os
from datetime import datetime

os.makedirs("saved_runs", exist_ok=True)
with open(os.path.join("saved_runs", f"ga{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.pkl"), "wb") as f:
    pickle.dump(ga, f) 

## Fallback functions for when something went wrong

### Access Best Solution from Disk

In [ ]:
import os
import glob
import evolutionary.image_processing as ip
from PIL import Image

num_generations = 42  # Set this to the number of generations you ran (if you didn't finish)
generation_dir = os.path.join(ip.RESULTS_FOLDER, f"{num_generations}")
image_files = glob.glob(os.path.join(generation_dir, "*.png"))
image_files.sort(key=ip.fitness_filename_sorting_key, reverse=True)
print(image_files[0])
Image.open(image_files[0])

### ffmpeg is not installed, create GIF instead

In [ ]:
from evolutionary.image_processing import create_animation_from_generations_pil
video_loc = create_animation_from_generations_pil(num_generations)
print(video_loc)
from IPython.display import Video
Video(filename=video_loc) 